In [1]:
from pytorch_tabnet.tab_model import TabNetRegressor

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np
np.random.seed(0)


import os
import wget
from pathlib import Path


%load_ext autoreload

%autoreload 2

# Download census-income dataset

In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
dataset_name = 'census-income'
out = Path(os.getcwd()+'/data/'+dataset_name+'.csv')

In [3]:
out.parent.mkdir(parents=True, exist_ok=True)
if out.exists():
    print("File already exists.")
else:
    print("Downloading file...")
    wget.download(url, out.as_posix())

File already exists.


# Load data and split

In [4]:
train = pd.read_csv(out)
target = ' <=50K'
if "Set" not in train.columns:
    train["Set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(train.shape[0],))

train_indices = train[train.Set=="train"].index
valid_indices = train[train.Set=="valid"].index
test_indices = train[train.Set=="test"].index

# Simple preprocessing

Label encode categorical features and fill empty cells.

In [5]:
categorical_columns = []
categorical_dims =  {}
for col in train.columns[train.dtypes == object]:
    print(col, train[col].nunique())
    l_enc = LabelEncoder()
    train[col] = train[col].fillna("VV_likely")
    train[col] = l_enc.fit_transform(train[col].values)
    categorical_columns.append(col)
    categorical_dims[col] = len(l_enc.classes_)

for col in train.columns[train.dtypes == 'float64']:
    train.fillna(train.loc[train_indices, col].mean(), inplace=True)

 State-gov 9
 Bachelors 16
 Never-married 7
 Adm-clerical 15
 Not-in-family 6
 White 5
 Male 2
 United-States 42
 <=50K 2
Set 3


# Define categorical features for categorical embeddings

In [6]:
unused_feat = ['Set']

features = [ col for col in train.columns if col not in unused_feat+[target]] 

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

# define your embedding sizes : here just a random choice
cat_emb_dim = [5, 4, 3, 6, 2, 2, 1, 10]

# Network parameters

In [7]:
clf = TabNetRegressor(cat_dims=cat_dims, cat_emb_dim=cat_emb_dim, cat_idxs=cat_idxs)

c:\Users\User\OneDrive - Nanyang Technological University\Desktop\NTU\Y1S1\SC1003- Intro to Computer Thinking\C Coding\SC1015_Mini_Proj\tabnet1\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


# Training

### We will simulate 5 targets here to perform multi regression without changing anything!

In [8]:
n_targets = 8

X_train = train[features].values[train_indices]
y_train = train[target].values[train_indices]
y_train = np.transpose(np.tile(y_train, (n_targets,1)))

X_valid = train[features].values[valid_indices]
y_valid = train[target].values[valid_indices]
y_valid = np.transpose(np.tile(y_valid, (n_targets,1)))

X_test = train[features].values[test_indices]
y_test = train[target].values[test_indices]
y_test = np.transpose(np.tile(y_test, (n_targets,1)))

In [11]:
max_epochs = 20 if not os.getenv("CI", False) else 2

In [12]:
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['rmsle', 'mae', 'rmse', 'mse'],
    max_epochs=max_epochs,
    patience=50,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    drop_last=False
) 

epoch 0  | loss: 0.36513 | train_rmsle: 0.48274 | train_mae: 0.59701 | train_rmse: 0.74438 | train_mse: 0.5541  | valid_rmsle: 0.47758 | valid_mae: 0.58689 | valid_rmse: 0.72615 | valid_mse: 0.52729 |  0:00:05s
epoch 1  | loss: 0.14484 | train_rmsle: 0.39332 | train_mae: 0.45489 | train_rmse: 0.53283 | train_mse: 0.2839  | valid_rmsle: 0.39027 | valid_mae: 0.44901 | valid_rmse: 0.53371 | valid_mse: 0.28485 |  0:00:10s
epoch 2  | loss: 0.12887 | train_rmsle: 0.37111 | train_mae: 0.41208 | train_rmse: 0.50125 | train_mse: 0.25126 | valid_rmsle: 0.36573 | valid_mae: 0.40146 | valid_rmse: 0.4912  | valid_mse: 0.24128 |  0:00:15s
epoch 3  | loss: 0.12121 | train_rmsle: 0.33641 | train_mae: 0.35368 | train_rmse: 0.4555  | train_mse: 0.20748 | valid_rmsle: 0.33051 | valid_mae: 0.34521 | valid_rmse: 0.45348 | valid_mse: 0.20564 |  0:00:23s
epoch 4  | loss: 0.11875 | train_rmsle: 0.2976  | train_mae: 0.30313 | train_rmse: 0.40168 | train_mse: 0.16134 | valid_rmsle: 0.29105 | valid_mae: 0.29415 

KeyboardInterrupt: 

In [ ]:
preds = clf.predict(X_test)

test_mse = mean_squared_error(y_pred=preds, y_true=y_test)

print(f"BEST VALID SCORE FOR {dataset_name} : {clf.best_cost}")
print(f"FINAL TEST SCORE FOR {dataset_name} : {test_mse}")

# Global explainability : feat importance summing to 1

In [ ]:
clf.feature_importances_

# Local explainability and masks

In [ ]:
explain_matrix, masks = clf.explain(X_test)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20,20))

for i in range(3):
    axs[i].imshow(masks[i][:50])
    axs[i].set_title(f"mask {i}")


# XGB : unfortunately this is still not possible with XGBoost

https://github.com/dmlc/xgboost/issues/2087